In [1]:
from pycalphad import Database, Model, variables as v
from pycalphad import equilibrium
import numpy as np
from pycalphad.tests.datasets import *
dbf = Database('alzn_mey.tdb')

In [9]:
v.T.default_value = 800
eq2 = equilibrium(dbf, ['AL', 'ZN', 'VA'], ['FCC_A1', 'HCP_A3', 'LIQUID'],
                 {v.X('ZN'): 0.25, v.NP('LIQUID'): 0.2817,
                  v.P: 1e5}, verbose=True)
print(eq2)

Components: AL VA ZN
Phases: FCC_A1 
HCP_A3 
LIQUID 
[done]
('Redundant phase:', CompositionSet(LIQUID, [ 0.31170852  0.68829148], NP=1e-06, GM=-40399.43865636517))
Removing CompositionSet(LIQUID, [ 0.31170852  0.68829148], NP=nan, GM=-40399.43865636517)
Trying to improve poor solution
Calculation Failed:  OrderedDict([('N', array(1.0)), ('NP_LIQUID', array(0.2817)), ('P', array(100000.0)), ('X_ZN', array(0.25))]) b'Maximum number of iterations exceeded (can be specified by an option).'
Chemical Potentials [ -6.62814480e+08  -3.19404600e+09]
[  9.99999990e-12   1.00000000e-16   3.60871787e-20   1.33332658e-11
   4.00006048e-11   1.33335050e-11   3.99984522e-11   1.39217789e-11
   3.54988823e-11]
[  1.00000000e+00   1.00000000e+05   2.77106728e+08   7.50003790e-01
   2.49996210e-01   7.49990336e-01   2.50009664e-01   7.18300000e-01
   2.81700000e-01]
Status: -1 b'Maximum number of iterations exceeded (can be specified by an option).'
<xarray.Dataset>
Dimensions:    (N: 1, NP_LIQUID: 1, 